In [7]:
import numpy as np
from qutip import basis, ket2dm, tensor, expect, fidelity, qeye

# Define the states
H = basis(2,0)
V = basis(2,1)

## Maximally entangled EPR pairs:
phi_plus = (tensor(H, H) + tensor(V, V)).unit()

## Maximally entangled EPR pairs with mixture:
rho = 0.7*ket2dm(phi_plus) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

## Non-maximally entangled pair:
phi = (1/np.sqrt(5)*tensor(H,H) + 2/np.sqrt(5)*tensor(V,V)).unit()


## Non-maximally entangled pair with mixture:
rho_prime = 0.7*ket2dm(phi) + 0.15*ket2dm(tensor(H, V)) + 0.15*ket2dm(tensor(V, H))

### Simulation of photon generation

In [2]:
def simulate_detection(rate, eff, dark_rate, dead_time, totol_time):
    t, counts = 0, 0
    dead = False
    while t < totol_time:
        if dead:
            t+= dead_time
            dead = False
        else:  
            photon_interval = np.random.exponential(1/rate)
            dark_interval = np.random.exponential(1/dark_rate)
            dt = min(photon_interval, dark_interval)
            t += dt
            if np.random.rand() < eff:
                counts += 1
                dead = True
    return counts

### Basis selection and Bell measurement

In [9]:
proj = lambda theta: (np.cos(theta)*H + np.sin(theta)*V).unit()

def compute_expectation(rho, theta_a, theta_b):
    A = ket2dm(proj(theta_a))
    B = ket2dm(proj(theta_b))
    AB = tensor(A, B)
    E = expect(AB, rho) + expect(tensor(qeye(2) - A, qeye(2) - B), rho) - \
        expect(tensor(A, qeye(2) - B), rho) - expect(tensor(qeye(2) - A, B), rho)
    return E

### Compute metrics
- Total Count Rate: Count all photons detected (including dark counts) per second
- Coincidence Rate: Count events detected simultaneously (within 1ns window)
- Fidelity: Calculate using QuTip
- Bell S parameter

In [10]:
states = {'Phi+': phi_plus, 'rho': rho, 'phi': phi, 'rho_prime': rho_prime}
results = {}

for name, state in states.items():
    counts_A = simulate_detection(15000, 0.1, 1000, 4e-6, 30)
    counts_B = simulate_detection(15000, 0.1, 1000, 4e-6, 30)

    coincidence_rate = min(counts_A, counts_B) /30

    F = fidelity(state, ket2dm(phi_plus))
    E1 = compute_expectation(state, 0, np.pi/8) # 0, 22.5
    E2 = compute_expectation(state, 0, 3*np.pi/8) # 0, 67.5
    E3 = compute_expectation(state, np.pi/4, np.pi/8) # 45, 22.5
    E4 = compute_expectation(state, np.pi/4, 3*np.pi/8) # 45, 67.5

    S = E1 - E2 + E3 + E4

    results[name] = {
        'Total Counts (Alice)': counts_A/30,
        'Totol Counts (Bob)': counts_B/30,
        'Coincidence Rate': coincidence_rate,
        'Fidelity': F,
        'Bell S Value': S
    }

print(results)

{'Phi+': {'Total Counts (Alice)': 1594.8333333333333, 'Totol Counts (Bob)': 1588.7666666666667, 'Coincidence Rate': 1588.7666666666667, 'Fidelity': np.float64(0.9999999999999997), 'Bell S Value': 2.82842712474619}, 'rho': {'Total Counts (Alice)': 1594.7333333333333, 'Totol Counts (Bob)': 1598.3, 'Coincidence Rate': 1594.7333333333333, 'Fidelity': np.float64(0.8366600265340758), 'Bell S Value': 1.5556349186104041}, 'phi': {'Total Counts (Alice)': 1585.8666666666666, 'Totol Counts (Bob)': 1594.8, 'Coincidence Rate': 1585.8666666666666, 'Fidelity': np.float64(0.9486832980505141), 'Bell S Value': 2.5455844122715723}, 'rho_prime': {'Total Counts (Alice)': 1581.1333333333334, 'Totol Counts (Bob)': 1581.4666666666667, 'Coincidence Rate': 1581.1333333333334, 'Fidelity': np.float64(0.7937253956848632), 'Bell S Value': 1.357645019878172}}
